In [1]:
%%time
import asyncio
from requests_html import AsyncHTMLSession
asession = AsyncHTMLSession()
import json
import pprint
import pandas as pd
import re

CPU times: total: 1.53 s
Wall time: 2.7 s


In [1]:
# Fetching data through API request  
r = await asession.get('https://www.zara.com/in/en/categories?ajax=true')
r.status_code

NameError: name 'asession' is not defined

In [3]:
json_string = r.text
data = json.loads(json_string)
# List to collect all the category codes
my_list = []

In [4]:
%%time
# creating urls for each page of the site
def invoke(id,name,sec_name,seo_id,extra):
    url = f"https://www.zara.com/in/en/category/{id}/products?regionGroupId=69&ajax=true"
    my_list.append([id,name,sec_name,seo_id, extra,url])


cat = data['categories']    
for i in cat:
    extra = 0
    # 4 main sc_1 (WOMAN, MAN, KIDS, BEAUTY)
    sc_1 = i['subcategories']
    for j in sc_1:
        # BEAUTY has only one sub-section
        id =j['id']
        name = j['name']
        sec_name = j.get("sectionName")
        seo_id = j.get('seo', {}).get('seoCategoryId', None)
        if(i['name'] == 'BEAUTY'):
            invoke(id,name,sec_name,seo_id, extra)
        elif (j['subcategories'] != []):
            sc_2 = j['subcategories']
            for k in sc_2:
                # MAN, WOMAN last subs
                id = k['id']
                m_name = k['name']
                seo_id = k.get('seo', {}).get('seoCategoryId', None)
                invoke(id,m_name,sec_name,seo_id,extra)
                if (k['subcategories'] != []):
                    # KIDS collection subcategories
                    sc_3 = k['subcategories']
                    for k_sc in sc_3:                                        
                        id = k_sc['id']
                        name = k_sc['name']
                        seo_id = k_sc.get('seo', {}).get('seoCategoryId', None)
                        invoke(id,name,sec_name,seo_id,extra=m_name)        

CPU times: total: 15.6 ms
Wall time: 4.53 ms


In [5]:
%%time
len(my_list)
df = pd.DataFrame(my_list)
df.to_csv('zara_code_list')

CPU times: total: 15.6 ms
Wall time: 30 ms


In [6]:
%%time
#improve
regex = r'\+ Info|JOIN LIFE|GIFT CARD|info'
for i in my_list:
    for j in i:
        if isinstance(j, str) and bool(re.search(regex, str(j), re.IGNORECASE)):
            my_list.remove(i)

print(len(my_list))

1228
CPU times: total: 31.2 ms
Wall time: 30.1 ms


In [7]:
sections = ['WOMAN', 'MAN', 'KID', 'BEAUTY']
product_list = []
links = []

for i in sections:
    print(i)
    for j in my_list:        
        if i == j[2]:
            val = j[5]
            c = j[0]
            # Collecting all responses of the urls
            r = await asession.get(val)
            print(r.status_code)
            try:
                r.raise_for_status()  # Raise an error for bad HTTP responses
                json_string = r.text
                data = json.loads(json_string)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
            except Exception as e:
                print(f"HTTP request error: {e}")
            product_list.append(data)
            links.append(c)     

WOMAN
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
MAN
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
20

In [8]:
import os


output_directory = "C:\Json files of zara 1200+responses"

os.makedirs(output_directory, exist_ok=True)

# Save each JSON object in a separate file
for i, json_data in enumerate(product_list):
    file_path = os.path.join(output_directory, f'zarajson_{i}_{links[i]}.json')
    with open(file_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=2)

print(f"All {len(product_list)} JSON files have been saved to {output_directory}")

All 1194 JSON files have been saved to C:\Json files of zara 1200+responses


In [9]:
%%time

def call(i):
    file_path = f"C:\Json files of zara 1200+responses\zarajson_{i}_{links[i]}.json"
    
    # Read the JSON file
    with open(file_path, 'r') as json_file:
        response = json.load(json_file)    
    # Process the JSON data as needed
    print(f"Processing {file_path}:")
    return response

# parsing every response and then storing them into individual files
def extract_info_recursive_generator(data):
    if isinstance(data, list):
        for item in data:
            yield from extract_info_recursive_generator(item)
    elif isinstance(data, dict):
        xmedia = data.get("commercialComponents", {})
        for entry in xmedia:
            yield {
                "name": entry.get("name"),
                "description": entry.get("description"),
                "price": entry.get("price"),
                "sectionName": entry.get("sectionName"),
                "familyName": entry.get("familyName"),
                "subfamilyName": entry.get("subfamilyName"),
                "id": entry.get("detail", {}).get("colors", [])[0].get("id"),
                "productId": entry.get("detail", {}).get("colors", [])[0].get("productId"),
                "color": entry.get("detail", {}).get("colors", [])[0].get("name")
            }

        for value in data.values():
            yield from extract_info_recursive_generator(value)

for i in range(1194):
    data = call(i)
    extracted_info_generator = extract_info_recursive_generator(data)
    extracted_info_list = list(extracted_info_generator)

    output_directory = "C:\Zara product details"
    os.makedirs(output_directory, exist_ok=True)

    # Save the extracted information to a separate JSON file for each input file
    output_file_path = os.path.join(output_directory, f"parsed_output_{i}.json")
    with open(output_file_path, 'w') as output_file:
        json.dump(extracted_info_list, output_file)


Processing C:\Json files of zara 1200+responses\zarajson_0_2290847.json:
Processing C:\Json files of zara 1200+responses\zarajson_1_2290837.json:
Processing C:\Json files of zara 1200+responses\zarajson_2_2290794.json:
Processing C:\Json files of zara 1200+responses\zarajson_3_2290827.json:
Processing C:\Json files of zara 1200+responses\zarajson_4_2290830.json:
Processing C:\Json files of zara 1200+responses\zarajson_5_2290803.json:
Processing C:\Json files of zara 1200+responses\zarajson_6_2290820.json:
Processing C:\Json files of zara 1200+responses\zarajson_7_2290866.json:
Processing C:\Json files of zara 1200+responses\zarajson_8_2290793.json:
Processing C:\Json files of zara 1200+responses\zarajson_9_2291012.json:
Processing C:\Json files of zara 1200+responses\zarajson_10_2291002.json:
Processing C:\Json files of zara 1200+responses\zarajson_11_2290993.json:
Processing C:\Json files of zara 1200+responses\zarajson_12_2290997.json:
Processing C:\Json files of zara 1200+responses\

In [10]:
# Processing every output file and combining all of it into one single large dataframe
result_df = pd.DataFrame()
for i in range(1194):        
    # Specify the directory where the JSON files are saved
    file_path = f"C:\Zara product details\parsed_output_{i}.json"
    # Read the JSON file
    with open(file_path, 'r') as json_file:
        response = json.load(json_file)    
    # Process the JSON data as needed
    print(f"Processing {file_path}:")
    df = pd.DataFrame(response)
    df['category_id'] = links[i]
    result_df = pd.concat([result_df, df], ignore_index=True, sort=False)

Processing C:\Zara product details\parsed_output_0.json:
Processing C:\Zara product details\parsed_output_1.json:
Processing C:\Zara product details\parsed_output_2.json:
Processing C:\Zara product details\parsed_output_3.json:
Processing C:\Zara product details\parsed_output_4.json:
Processing C:\Zara product details\parsed_output_5.json:
Processing C:\Zara product details\parsed_output_6.json:
Processing C:\Zara product details\parsed_output_7.json:
Processing C:\Zara product details\parsed_output_8.json:
Processing C:\Zara product details\parsed_output_9.json:
Processing C:\Zara product details\parsed_output_10.json:
Processing C:\Zara product details\parsed_output_11.json:
Processing C:\Zara product details\parsed_output_12.json:
Processing C:\Zara product details\parsed_output_13.json:
Processing C:\Zara product details\parsed_output_14.json:
Processing C:\Zara product details\parsed_output_15.json:
Processing C:\Zara product details\parsed_output_16.json:
Processing C:\Zara produ

C:\Users\madan\AppData\Local\Temp\ipykernel_6660\256223566.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, df], ignore_index=True, sort=False)


Processing C:\Zara product details\parsed_output_325.json:
Processing C:\Zara product details\parsed_output_326.json:
Processing C:\Zara product details\parsed_output_327.json:
Processing C:\Zara product details\parsed_output_328.json:
Processing C:\Zara product details\parsed_output_329.json:
Processing C:\Zara product details\parsed_output_330.json:
Processing C:\Zara product details\parsed_output_331.json:
Processing C:\Zara product details\parsed_output_332.json:
Processing C:\Zara product details\parsed_output_333.json:
Processing C:\Zara product details\parsed_output_334.json:
Processing C:\Zara product details\parsed_output_335.json:
Processing C:\Zara product details\parsed_output_336.json:
Processing C:\Zara product details\parsed_output_337.json:
Processing C:\Zara product details\parsed_output_338.json:
Processing C:\Zara product details\parsed_output_339.json:
Processing C:\Zara product details\parsed_output_340.json:
Processing C:\Zara product details\parsed_output_341.jso

In [17]:
# Total rows = 102147
result_df.tail(30)

,name,description,price,sectionName,familyName,subfamilyName,id,productId,color,category_id
102118,EYE BRUSH #2,Compact and flat eye brush specially designed ...,59000.0,WOMAN,BORLAS Y BROCHAS,COSM-COMPLEMENT,250,277535300.0,,1881287
102119,,,NaN,None,None,None,927,288663324.0,927,1881287
102120,EYE BRUSH #1,Small eye brush designed to blend or apply eye...,59000.0,WOMAN,BORLAS Y BROCHAS,COSM-COMPLEMENT,250,277546910.0,,1881287
102121,,,59000.0,WOMAN,None,None,214,290675904.0,214,1881287
102122,EYELASH CURLER,Allows the eyelashes to be curled and raised i...,89000.0,WOMAN,COMPLEMENTOS,COSM-COMPLEMENT,800,290431181.0,2-800,1881287
102123,,,NaN,None,None,None,104,290137851.0,104,1881287
102124,EYELASH CURLER,Allows the eyelashes to be curled and raised i...,89000.0,WOMAN,COMPLEMENTOS,COSM-COMPLEMENT,800,290431182.0,2-800,1881287
102125,EYELASH CURLER,Allows the eyelashes to be curled and raised i...,89000.0,WOMAN,COMPLEMENTOS,COSM-COMPLEMENT,800,290431180.0,2-800,1881287
102126,,,NaN,None,None,None,989,290138809.0,989,2298138
102127,,,NaN,None,None,None,990,290139307.0,990,2298138
